In [15]:
import gymnasium as gym
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from datetime import datetime
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import RBFSampler

print(f"Gym version: {gym.__version__}")
print(f"Tensorflow version: {tf.__version__}")

Gym version: 0.29.1
Tensorflow version: 2.16.1


In [3]:
#class SGDRegressor:
#  def __init__(self, D):
#    self.w = np.random.randn(D) / np.sqrt(D)
#    self.lr = 0.1
#
#  def partial_fit(self, X, Y):
#    self.w += self.lr*(Y - X.dot(self.w)).dot(X)
#
#  def predict(self, X):
#    return X.dot(self.w)
#

In [46]:
class SGDRegressor:
    def __init__(self, D):
        print("Hello TensorFlow!")
        lr = 0.1
    
        inputs = tf.keras.Input(shape=(D,), dtype=tf.float32, name='X')
        outputs = tf.keras.layers.Dense(1, activation="linear")(inputs)
        self.model = tf.keras.Model(inputs=inputs, outputs=outputs)
        self.optimizer = tf.keras.optimizers.SGD(learning_rate=lr)

    def partial_fit(self, X, Y):
        with tf.GradientTape() as tape:
            y = self.model(X)
        loss = tf.reduce_mean(tf.square(tf.subtract(y, Y)))
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply(grads, self.model.trainable_variables)

    def predict(self, X):
        return self.model(X)


In [41]:
class FeatureTransformer:
  def __init__(self, env):
    # observation_examples = np.array([env.observation_space.sample() for x in range(10000)])
    # NOTE!! state samples are poor, b/c you get velocities --> infinity
    observation_examples = np.random.random((20000, 4))*2 - 1
    scaler = StandardScaler()
    scaler.fit(observation_examples)

    # Used to converte a state to a featurizes represenation.
    # We use RBF kernels with different variances to cover different parts of the space
    featurizer = FeatureUnion([
            ("rbf1", RBFSampler(gamma=0.05, n_components=1000)),
            ("rbf2", RBFSampler(gamma=1.0, n_components=1000)),
            ("rbf3", RBFSampler(gamma=0.5, n_components=1000)),
            ("rbf4", RBFSampler(gamma=0.1, n_components=1000))
            ])
    feature_examples = featurizer.fit_transform(scaler.transform(observation_examples))

    self.dimensions = feature_examples.shape[1]
    self.scaler = scaler
    self.featurizer = featurizer

  def transform(self, observations):
    scaled = self.scaler.transform(observations)
    return self.featurizer.transform(scaled)


In [42]:
class Model:
  def __init__(self, env, feature_transformer):
    self.env = env
    self.models = []
    self.feature_transformer = feature_transformer
    for i in range(env.action_space.n):
      model = SGDRegressor(feature_transformer.dimensions)
      self.models.append(model)

  def predict(self, s):
    X = self.feature_transformer.transform(np.atleast_2d(s))
    result = np.stack([m.predict(X) for m in self.models]).T
    return result

  def update(self, s, a, G):
    X = self.feature_transformer.transform(np.atleast_2d(s))
    self.models[a].partial_fit(X, [G])

  def sample_action(self, s, eps):
    if np.random.random() < eps:
      return self.env.action_space.sample()
    else:
      return np.argmax(self.predict(s))


In [43]:
def play_one(env, model, eps, gamma):
  observation, _ = env.reset()
  done = False
  totalreward = 0
  iters = 0
  while not done and iters < 2000:
    # if we reach 2000, just quit, don't want this going forever
    # the 200 limit seems a bit early
    action = model.sample_action(observation, eps)
    prev_observation = observation
    observation, reward, done, info, _ = env.step(action)

    if done:
      reward = -200

    # update the model
    next = model.predict(observation)
    # print(next.shape)
    #assert(next.shape == (1, env.action_space.n))
    G = reward + gamma*np.max(next)
    model.update(prev_observation, action, G)

    if reward == 1: # if we changed the reward to -200
      totalreward += reward
    iters += 1

  return totalreward


In [44]:
def plot_running_avg(totalrewards):
  N = len(totalrewards)
  running_avg = np.empty(N)
  for t in range(N):
    running_avg[t] = totalrewards[max(0, t-100):(t+1)].mean()
  plt.plot(running_avg)
  plt.title("Running Average")
  plt.show()


In [47]:
env = gym.make('CartPole-v1')
ft = FeatureTransformer(env)
model = Model(env, ft)
gamma = 0.99

N = 500
totalrewards = np.empty(N)
costs = np.empty(N)
for n in range(N):
    eps = 1.0/np.sqrt(n+1)
    totalreward = play_one(env, model, eps, gamma)
    totalrewards[n] = totalreward
    if n % 100 == 0:
        print("episode:", n, "total reward:", totalreward, "eps:", eps, "avg reward (last 100):", totalrewards[max(0, n-100):(n+1)].mean())

print("avg reward for last 100 episodes:", totalrewards[-100:].mean())
print("total steps:", totalrewards.sum())

plt.plot(totalrewards)
plt.title("Rewards")
plt.show()

plot_running_avg(totalrewards)


Hello TensorFlow!
Hello TensorFlow!


ValueError: No gradients provided for any variable.